# Lecture 10—Text Processing

Version 1.0. Prepared by [Makzan](https://makzan.net). Updated at 2021 March.

In this lecture, we will explore reading and writing plain text files.

We will learn:

- [Process text using regular expression](#Process-Text-with-Regular-Expression)
- [Combining DataFrame and Regex](#Combining-DataFrame-and-Regex)

## Process Text with Regular Expression

We can use Regular Expression to process text with patterns.

In [13]:
with open("sample-years.txt") as file_obj:
    lines = file_obj.read().splitlines()
    
lines

['Steven Hawking was born in 1942.',
 'Albert Einstein was born in 1879',
 'Albert Einstein won Nobel Prize in 1921.',
 'Stephen Curry wear No. 30.',
 'Stephen Curry went into NBA in 2009',
 'Stephen Curry won NBA MVP in 2015 and 2016.',
 'Micheal Jordan was born in 1963.']

The following code finds all the years in the text document.

In [14]:
import re

for line in lines:
    pattern = '\d{4}'
    print(re.findall(pattern, line))


['1942']
['1879']
['1921']
[]
['2009']
['2015', '2016']
['1963']


What if we only want the first year found? 

Let’s try using `[0]` to get the first result for each line. And then, we have an error:

In [18]:
import re

for line in lines:
    pattern = '\d{4}'
    print(re.findall(pattern, line)[0])


1942
1879
1921


IndexError: list index out of range

The error occurs because there is one line that failed to find any year result.

We can ensure there is empty result by searching the ending of line too. This result in an extra result in every reuslt:

In [20]:
import re

for line in lines:
    pattern = '\d{4}|$'
    print(re.findall(pattern, line))


['1942', '']
['1879', '']
['1921', '']
['']
['2009', '']
['2015', '2016', '']
['1963', '']


But it is useful if we need to ensure the first result.

In [21]:
import re

for line in lines:
    pattern = '\d{4}|$'
    print(re.findall(pattern, line)[0])


1942
1879
1921

2009
2015
1963


The following code finds all the names in the text document

In [15]:
import re

for line in lines:
    pattern = '[A-Z][a-z]* [A-Z][a-z]*'
    print(re.findall(pattern, line))


['Steven Hawking']
['Albert Einstein']
['Albert Einstein', 'Nobel Prize']
['Stephen Curry']
['Stephen Curry']
['Stephen Curry', 'A M']
['Micheal Jordan']


In [16]:
import re

for line in lines:
    pattern = '[A-Z][a-z]+ [A-Z][a-z]+'
    print(re.findall(pattern, line))


['Steven Hawking']
['Albert Einstein']
['Albert Einstein', 'Nobel Prize']
['Stephen Curry']
['Stephen Curry']
['Stephen Curry']
['Micheal Jordan']


In [17]:
import re

for line in lines:
    pattern = '^[A-Z][a-z]+ [A-Z][a-z]+'
    print(re.findall(pattern, line))


['Steven Hawking']
['Albert Einstein']
['Albert Einstein']
['Stephen Curry']
['Stephen Curry']
['Stephen Curry']
['Micheal Jordan']


You can read more examples of [using Regular Expression on Programiz.com](https://www.programiz.com/python-programming/regex).

## Combining DataFrame and Regex

We can combine data frame and regular expression to perform column-based operation to all data at once.

In [3]:
import pandas as pd

df = pd.read_csv('sample-years.txt', header=None, names=['Original Text'])

df

,Original Text
0,Steven Hawking was born in 1942.
1,Albert Einstein was born in 1879
2,Albert Einstein won Nobel Prize in 1921.
3,Stephen Curry wear No. 30.
4,Stephen Curry went into NBA in 2009
5,Stephen Curry won NBA MVP in 2015 and 2016.
6,Micheal Jordan was born in 1963.


Now that we loaded the text into a column, we can create a new column that applies our own transformation.

We define the function that find first year and name given the string parameter input.

In [4]:
def find_first_year(string):
    pattern = '\d{4}|$'
    return re.findall(pattern, string)[0]

def find_first_name(string):
    pattern = '^[A-Z][a-z]+ [A-Z][a-z]+|$'
    return re.findall(pattern, string)[0]

In [5]:
import re

df["Years"] = df['Original Text'].apply(find_first_year)
df["Name"] = df['Original Text'].apply(find_first_name)

df

,Original Text,Years,Name
0,Steven Hawking was born in 1942.,1942,Steven Hawking
1,Albert Einstein was born in 1879,1879,Albert Einstein
2,Albert Einstein won Nobel Prize in 1921.,1921,Albert Einstein
3,Stephen Curry wear No. 30.,,Stephen Curry
4,Stephen Curry went into NBA in 2009,2009,Stephen Curry
5,Stephen Curry won NBA MVP in 2015 and 2016.,2015,Stephen Curry
6,Micheal Jordan was born in 1963.,1963,Micheal Jordan


In [6]:
df.sort_values(by="Years")

,Original Text,Years,Name
3,Stephen Curry wear No. 30.,,Stephen Curry
1,Albert Einstein was born in 1879,1879,Albert Einstein
2,Albert Einstein won Nobel Prize in 1921.,1921,Albert Einstein
0,Steven Hawking was born in 1942.,1942,Steven Hawking
6,Micheal Jordan was born in 1963.,1963,Micheal Jordan
4,Stephen Curry went into NBA in 2009,2009,Stephen Curry
5,Stephen Curry won NBA MVP in 2015 and 2016.,2015,Stephen Curry


In [7]:
df.sort_values(by="Name")

,Original Text,Years,Name
1,Albert Einstein was born in 1879,1879,Albert Einstein
2,Albert Einstein won Nobel Prize in 1921.,1921,Albert Einstein
6,Micheal Jordan was born in 1963.,1963,Micheal Jordan
3,Stephen Curry wear No. 30.,,Stephen Curry
4,Stephen Curry went into NBA in 2009,2009,Stephen Curry
5,Stephen Curry won NBA MVP in 2015 and 2016.,2015,Stephen Curry
0,Steven Hawking was born in 1942.,1942,Steven Hawking


## Example: Counting words in Alice in Wonderland

What we can do with `count`? For example, we can count how many times a word appear in a paragraph. For instance:

In [4]:
paragraph = "Hello World. Welcome to the Python course. Here is what you can do with Python."

paragraph.count("Python")

The following is chapter 1 of [Alice in the Wonderland](http://www.gutenberg.org/ebooks/928), extracted from [Gutenberg.org](http://www.gutenberg.org/files/928/928-h/928-h.htm).

In [5]:
chapter_1 = """CHAPTER I

Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.

There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.

In another moment down went Alice after it, never once considering how in the world she was to get out again.

The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.

Either the well was very deep, or she fell very slowly, for she had plenty of time as she went down to look about her and to wonder what was going to happen next. First, she tried to look down and make out what she was coming to, but it was too dark to see anything; then she looked at the sides of the well, and noticed that they were filled with cupboards and book-shelves; here and there she saw maps and pictures hung upon pegs. She took down a jar from one of the shelves as she passed; it was labelled 'ORANGE MARMALADE', but to her great disappointment it was empty: she did not like to drop the jar for fear of killing somebody, so managed to put it into one of the cupboards as she fell past it.

'Well!' thought Alice to herself, 'after such a fall as this, I shall think nothing of tumbling down stairs! How brave they'll all think me at home! Why, I wouldn't say anything about it, even if I fell off the top of the house!' (Which was very likely true.)

Down, down, down. Would the fall never come to an end! 'I wonder how many miles I've fallen by this time?' she said aloud. 'I must be getting somewhere near the centre of the earth. Let me see: that would be four thousand miles down, I think--' (for, you see, Alice had learnt several things of this sort in her lessons in the schoolroom, and though this was not a very good opportunity for showing off her knowledge, as there was no one to listen to her, still it was good practice to say it over) '--yes, that's about the right distance--but then I wonder what Latitude or Longitude I've got to?' (Alice had no idea what Latitude was, or Longitude either, but thought they were nice grand words to say.)

Presently she began again. 'I wonder if I shall fall right through the earth! How funny it'll seem to come out among the people that walk with their heads downward! The Antipathies, I think--' (she was rather glad there was no one listening, this time, as it didn't sound at all the right word) '--but I shall have to ask them what the name of the country is, you know. Please, Ma'am, is this New Zealand or Australia?' (and she tried to curtsey as she spoke--fancy curtseying as you're falling through the air! Do you think you could manage it?) 'And what an ignorant little girl she'll think me for asking! No, it'll never do to ask: perhaps I shall see it written up somewhere.'

Down, down, down. There was nothing else to do, so Alice soon began talking again. 'Dinah'll miss me very much to-night, I should think!' (Dinah was the cat.) 'I hope they'll remember her saucer of milk at tea-time. Dinah my dear! I wish you were down here with me! There are no mice in the air, I'm afraid, but you might catch a bat, and that's very like a mouse, you know. But do cats eat bats, I wonder?' And here Alice began to get rather sleepy, and went on saying to herself, in a dreamy sort of way, 'Do cats eat bats? Do cats eat bats?' and sometimes, 'Do bats eat cats?' for, you see, as she couldn't answer either question, it didn't much matter which way she put it. She felt that she was dozing off, and had just begun to dream that she was walking hand in hand with Dinah, and saying to her very earnestly, 'Now, Dinah, tell me the truth: did you ever eat a bat?' when suddenly, thump! thump! down she came upon a heap of sticks and dry leaves, and the fall was over.

Alice was not a bit hurt, and she jumped up on to her feet in a moment: she looked up, but it was all dark overhead; before her was another long passage, and the White Rabbit was still in sight, hurrying down it. There was not a moment to be lost: away went Alice like the wind, and was just in time to hear it say, as it turned a corner, 'Oh my ears and whiskers, how late it's getting!' She was close behind it when she turned the corner, but the Rabbit was no longer to be seen: she found herself in a long, low hall, which was lit up by a row of lamps hanging from the roof.

There were doors all round the hall, but they were all locked; and when Alice had been all the way down one side and up the other, trying every door, she walked sadly down the middle, wondering how she was ever to get out again.

Suddenly she came upon a little three-legged table, all made of solid glass; there was nothing on it except a tiny golden key, and Alice's first thought was that it might belong to one of the doors of the hall; but, alas! either the locks were too large, or the key was too small, but at any rate it would not open any of them. However, on the second time round, she came upon a low curtain she had not noticed before, and behind it was a little door about fifteen inches high: she tried the little golden key in the lock, and to her great delight it fitted!

Alice opened the door and found that it led into a small passage, not much larger than a rat-hole: she knelt down and looked along the passage into the loveliest garden you ever saw. How she longed to get out of that dark hall, and wander about among those beds of bright flowers and those cool fountains, but she could not even get her head though the doorway; 'and even if my head would go through,' thought poor Alice, 'it would be of very little use without my shoulders. Oh, how I wish I could shut up like a telescope! I think I could, if I only know how to begin.' For, you see, so many out-of-the-way things had happened lately, that Alice had begun to think that very few things indeed were really impossible.

There seemed to be no use in waiting by the little door, so she went back to the table, half hoping she might find another key on it, or at any rate a book of rules for shutting people up like telescopes: this time she found a little bottle on it, ('which certainly was not here before,' said Alice,) and round the neck of the bottle was a paper label, with the words 'DRINK ME' beautifully printed on it in large letters.

It was all very well to say 'Drink me,' but the wise little Alice was not going to do that in a hurry. 'No, I'll look first,' she said, 'and see whether it's marked "poison" or not'; for she had read several nice little histories about children who had got burnt, and eaten up by wild beasts and other unpleasant things, all because they would not remember the simple rules their friends had taught them: such as, that a red-hot poker will burn you if you hold it too long; and that if you cut your finger very deeply with a knife, it usually bleeds; and she had never forgotten that, if you drink much from a bottle marked 'poison,' it is almost certain to disagree with you, sooner or later.

However, this bottle was not marked 'poison,' so Alice ventured to taste it, and finding it very nice, (it had, in fact, a sort of mixed flavour of cherry-tart, custard, pine-apple, roast turkey, toffee, and hot buttered toast,) she very soon finished it off.


*     *     *     *     * 
*     *     *     * 
*     *     *     *     * 

'What a curious feeling!' said Alice; 'I must be shutting up like a telescope.'

And so it was indeed: she was now only ten inches high, and her face brightened up at the thought that she was now the right size for going through the little door into that lovely garden. First, however, she waited for a few minutes to see if she was going to shrink any further: she felt a little nervous about this; 'for it might end, you know,' said Alice to herself, 'in my going out altogether, like a candle. I wonder what I should be like then?' And she tried to fancy what the flame of a candle is like after the candle is blown out, for she could not remember ever having seen such a thing.

After a while, finding that nothing more happened, she decided on going into the garden at once; but, alas for poor Alice! when she got to the door, she found she had forgotten the little golden key, and when she went back to the table for it, she found she could not possibly reach it: she could see it quite plainly through the glass, and she tried her best to climb up one of the legs of the table, but it was too slippery; and when she had tired herself out with trying, the poor little thing sat down and cried.

'Come, there's no use in crying like that!' said Alice to herself, rather sharply; 'I advise you to leave off this minute!' She generally gave herself very good advice, (though she very seldom followed it), and sometimes she scolded herself so severely as to bring tears into her eyes; and once she remembered trying to box her own ears for having cheated herself in a game of croquet she was playing against herself, for this curious child was very fond of pretending to be two people. 'But it's no use now,' thought poor Alice, 'to pretend to be two people! Why, there's hardly enough of me left to make one respectable person!'

Soon her eye fell on a little glass box that was lying under the table: she opened it, and found in it a very small cake, on which the words 'EAT ME' were beautifully marked in currants. 'Well, I'll eat it,' said Alice, 'and if it makes me grow larger, I can reach the key; and if it makes me grow smaller, I can creep under the door; so either way I'll get into the garden, and I don't care which happens!'

She ate a little bit, and said anxiously to herself, 'Which way? Which way?', holding her hand on the top of her head to feel which way it was growing, and she was quite surprised to find that she remained the same size: to be sure, this generally happens when one eats cake, but Alice had got so much into the way of expecting nothing but out-of-the-way things to happen, that it seemed quite dull and stupid for life to go on in the common way.

So she set to work, and very soon finished off the cake."""

Given the chapter 1, how many times the word "Alice" appears? We can use `count` on the string directly.

In [6]:
chapter_1.count("Alice")

## Furthermore: Splitting the string into list

By splitting the string into list, we can further get the word counts. We can also use `Counter` to get the most frequent words.

Roughly, how many words are there in chapter 1?

We can split the string into a list of words by space.

In [7]:
# Split chapter 1 by space
words = chapter_1.split(" ")

In [8]:
len(words)

In [9]:
# Split chapter 1 by any whitespace
words = chapter_1.split()

In [10]:
len(words)

What is the top 20 most frequent words in chapter 1?

In [11]:
from collections import Counter 
Counter(words).most_common(20)

## Counting "Alice" in the list

Let’s count how many times "Alice" appearing in the list.

In [12]:
words.count("Alice")

Wait! We have different result because of the punctuation.

Here are all the punctuations, in Python `string` module.

In [13]:
import string
string.punctuation

In [14]:
import string

for punctuation in string.punctuation:
    chapter_1 = chapter_1.replace(punctuation, '')

words = chapter_1.split()
words.count("Alice")

More on [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) on pandas documentation.

## Summary

In this lecture, we explore text processing via Pandas. We also count the words in chapter 1 of Alice in Wonderland. Furthermore, we imported the string module and collections module to use more advanced built-in commands provided by Python.